In [575]:
import gspread
import schedule
import time
import os
from dateutil import parser

In [576]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import gradio as gr


In [577]:
import requests
from PIL import Image
import tempfile

In [599]:
def convert_google_drive_link_to_direct_url(link):
    # Convert Google Drive link to direct link format
    if 'drive.google.com' in link:
        file_id = link.split('/')[-2]  # Extract the file ID from the URL
        return f"https://drive.google.com/uc?id={file_id}"
    return link

def search_scheme(keyword):
    # Search logic for schemes based on the keyword
    search_result_df = stored_df_test_service[stored_df_test_service['Scheme'] == keyword]
    
    if not search_result_df.empty:
        pamphlet_link = search_result_df.iloc[0]['Pamphlet link']
        # Convert to direct link
        direct_link = convert_google_drive_link_to_direct_url(pamphlet_link)
    else:
        search_result_df = pd.DataFrame()  # Return an empty DataFrame if not found
        direct_link = None
    
    return search_result_df, direct_link  # Return the direct link instead of the image path


In [579]:
# Define the scope
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# Add your service account file
creds = ServiceAccountCredentials.from_json_keyfile_name(r"oddoo.json", scope)

In [580]:
client = gspread.authorize(creds)

In [581]:
# Function to get data from Google Sheets
def get_sheet_data(spreadsheet_id, worksheet_index):
    sheet = client.open(spreadsheet_id)
    worksheet = sheet.worksheet(worksheet_index)
    data = worksheet.get_all_values()
    header_row = data[0]
    data_rows = data[1:]  # The rest of the data
    # Convert to DataFrame
    df = pd.DataFrame(data_rows, columns=header_row)
    return df
    

In [582]:
spreadsheet_id = 'service_sheet'
worksheet_index='Sheet1'
stored_df_test_service = get_sheet_data(spreadsheet_id, worksheet_index)

In [583]:
stored_df_test_service['Days left'] = pd.to_numeric(stored_df_test_service['Days left'], errors='coerce')

In [584]:
stored_df_test_service.head()

,SR. NO.,Scheme,Eligibility Criteria,COMPANY TYPE,SECTOR,Benefits,Status,Deadline,Days left,Pamphlet link
0,1,Hackathon 4.0,Eligibility: Age 18-35 years (including both) ...,ALL,All Sector,Grant of up to INR 15 Lakhs,Live,10/10/2024,8,https://drive.google.com/file/d/1AHBhzCZgMqR3N...
1,2,Small Business Innovation Grant,Private Limited and LLP only can apply,"Private Limited, Limited Liability Partnership...","Agriculture, Biotechnology, Pharmaceuticals, H...",Grant in Aid Up to ₹ 50 Lakhs\n \n \n The Comp...,Live,30/10/2024,28,https://drive.google.com/file/d/1AHBhzCZgMqR3N...
2,3,Mission AI,"DIPP (Private Limited, LLP and Reg Partnership...","Private Limited, Limited Liability Partnership...","AI, Healthcare, Agriculture, Education, Enviro...",1. Initial submission and screening (Stage-1):...,Closed,CLOSED,999,https://drive.google.com/file/d/1AHBhzCZgMqR3N...
3,4,Funding Support (Guj),"LLP, Private Limited, & Partnership can apply ...","Private Limited, Limited Liability Partnership...",All Sector,Grant support: Up to ₹30 lakh\n Assistance to ...,Live,Rolling basis,999,https://drive.google.com/file/d/1AHBhzCZgMqR3N...
4,5,Ignition Grant,The startup should not be more than 2 years of...,"Private Limited, Limited Liability Partnership...",All Sector,The grant support can be provided – up to INR ...,Closed,CLOSED,999,https://drive.google.com/file/d/1AHBhzCZgMqR3N...


In [585]:
stored_df_test_service = stored_df_test_service[stored_df_test_service['Status'] == "Live"]

In [586]:
# stored_df_test_service = stored_df_test_service[stored_df_test_service['Days left'] < 0]

In [587]:
stored_df_test_service["COMPANY TYPE"] = stored_df_test_service["COMPANY TYPE"].str.split(',').apply(lambda x: [i.strip() for i in x])

In [588]:
stored_df_test_service["SECTOR"] = stored_df_test_service["SECTOR"].str.split(',').apply(lambda x: [i.strip() for i in x])

In [589]:
stored_df_test_service["Scheme"] = stored_df_test_service["Scheme"].str.strip()

In [590]:
unique_values = pd.Series([item for sublist in stored_df_test_service["COMPANY TYPE"] for item in sublist]).unique().tolist()

In [591]:
unique_values_sector = pd.Series([item for sublist in stored_df_test_service["SECTOR"] for item in sublist]).unique().tolist()

In [592]:
# ### Orignial - of no use now
# def filter_dataframe(selected_value , sector):
#     selected_value1 = 'ALL'
#     selected_sector_value = "All Sectors"
#     # filtered_df = stored_df_test_service[(stored_df_test_service["COMPANY TYPE"] == a)][["Scheme" , "Benefits"]] #& (stored_df_test_service["Criteria2"] == b)]
#     filtered_df = stored_df_test_service[stored_df_test_service["COMPANY TYPE"].apply(lambda x: selected_value in x)][["Scheme" , "Benefits" , "SECTOR"]]
#     filtered_df1 = stored_df_test_service[stored_df_test_service["COMPANY TYPE"].apply(lambda x: selected_value1 in x)][["Scheme" , "Benefits", "SECTOR"]]
#     filtered_df2 = pd.concat([filtered_df, filtered_df1]).drop_duplicates(subset='Scheme').reset_index(drop=True)
#     filtered_df3 = filtered_df2[filtered_df2["SECTOR"].apply(lambda x: selected_sector_value in x)][["Scheme" , "Benefits"]]
#     filtered_df4 = filtered_df2[filtered_df2["SECTOR"].apply(lambda x: sector in x)][["Scheme" , "Benefits"]]
#     filtered_df5 = pd.concat([filtered_df3, filtered_df4]).drop_duplicates(subset='Scheme').reset_index(drop=True)
#     return filtered_df5


In [593]:
def search_scheme(keyword):
    # Search logic for schemes based on the keyword
    search_result_df = stored_df_test_service[stored_df_test_service['Scheme'] == keyword]
    
    if not search_result_df.empty:
        pamphlet_link = search_result_df.iloc[0]['Pamphlet link']
        # Download and return the image file path if link exists
        image_filepath = download_image_from_url(pamphlet_link) if pamphlet_link else None
    else:
        image_filepath = None
    
    return search_result_df, image_filepath


In [594]:
# Get unique values for Criteria1 and Criteria2 for dropdowns
criteria1_options = unique_values
criteria2_options = unique_values_sector
criteria3_options = stored_df_test_service.Scheme.values.tolist()

In [595]:
### Final code for filters
def filter_dataframe(selected_value, sector):
    # Start with the full DataFrame
    filtered_df = stored_df_test_service[["SR. NO.", "Scheme", "Benefits", "SECTOR", "COMPANY TYPE" , "Deadline" , "Days left"]]

    # Apply Company Type filter
    if selected_value != 'ALL':
        # Include all company types when a specific type is selected
        filtered_df = filtered_df[(filtered_df["COMPANY TYPE"].apply(lambda x: selected_value in x)) | 
                                  (filtered_df["COMPANY TYPE"].apply(lambda x: 'ALL' in x))]

    # Apply Sector filter
    if sector != 'All Sector':
        # Include all sectors when a specific sector is selected
        filtered_df = filtered_df[(filtered_df["SECTOR"].apply(lambda x: sector in x)) | 
                                  (filtered_df["SECTOR"].apply(lambda x: 'All Sector' in x))]

    # Return the filtered DataFrame
    return filtered_df.sort_values(by='Days left', ascending=True)


In [600]:
### Final UI code 

with gr.Blocks() as ui:
    # Initial display of the entire DataFrame
    output = gr.DataFrame(value=stored_df_test_service[["SR. NO.", "Scheme", "Benefits", "SECTOR", "COMPANY TYPE", "Deadline" , "Days left"]].sort_values(by='Days left', ascending=True)
    , label="Filtered DataFrame")

    # Filters
    a_input = gr.Dropdown(label="Company Type", choices=criteria1_options, value='ALL')
    b_input = gr.Dropdown(label="Sector", choices=criteria2_options, value='All Sector')

    # When either filter changes, reapply filtering and update the output
    a_input.change(fn=filter_dataframe, inputs=[a_input, b_input], outputs=output)
    b_input.change(fn=filter_dataframe, inputs=[a_input, b_input], outputs=output)

    # Search functionality
    search_input = gr.Dropdown(label="Search Scheme", choices=criteria3_options)
    search_output = gr.DataFrame(label="Search Results")
    image_output = gr.Image(label="Pamphlet Image")

    search_button = gr.Button("Search")
    search_button.click(search_scheme, inputs=[search_input], outputs=[search_output, image_output])


In [597]:
# ### Orignal - UI code - of no use now
# with gr.Blocks() as ui:
#     a_input = gr.Dropdown(label="Company Type", choices=criteria1_options)
#     b_input = gr.Dropdown(label="Sector", choices=criteria2_options)
#     output = gr.DataFrame(label="Filtered DataFrame")

#     submit_button = gr.Button("Submit")
#     submit_button.click(filter_dataframe, inputs= [a_input , b_input], outputs=output)

#     search_input = gr.Dropdown(label="Search Scheme", choices=criteria3_options)
#     search_output = gr.DataFrame(label="Search Results")

#     # Submit Button for search-based filtering
#     search_button = gr.Button("Search")
#     search_button.click(search_scheme, inputs=[search_input], outputs=search_output)



In [601]:
# Launch the interface
ui.launch()

Running on local URL:  http://127.0.0.1:7917

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\Egnio\AppData\Roaming\Python\Python311\site-packages\httpx\_transports\default.py", line 69, in map_httpcore_exceptions
    yield
  File "C:\Users\Egnio\AppData\Roaming\Python\Python311\site-packages\httpx\_transports\default.py", line 373, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Egnio\AppData\Roaming\Python\Python311\site-packages\httpcore\_async\connection_pool.py", line 216, in handle_async_request
    raise exc from None
  File "C:\Users\Egnio\AppData\Roaming\Python\Python311\site-packages\httpcore\_async\connection_pool.py", line 196, in handle_async_request
    response = await connection.handle_async_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Egnio\AppData\Roaming\Python\Python311\site-packages\httpcore\_async\connection.py", line 101, in handle_async_request
    return await self._co

### Rough work

In [210]:
selected_value = 'Private Limited'
sector = 'All Sector'
selected_value1 = 'ALL'

In [209]:
# unique_values_sector

In [211]:
filtered_df = stored_df_test_service[stored_df_test_service["COMPANY TYPE"].apply(lambda x: selected_value in x)][["Scheme" , "Benefits" , "SECTOR"]]

In [213]:
filtered_df1 = stored_df_test_service[stored_df_test_service["COMPANY TYPE"].apply(lambda x: selected_value1 in x)][["Scheme" , "Benefits", "SECTOR"]]

In [217]:
filtered_df2 = pd.concat([filtered_df, filtered_df1])#.drop_duplicates().reset_index(drop=True)

In [219]:
filtered_df2 = filtered_df2.drop_duplicates(subset='Scheme').reset_index(drop=True)

In [220]:
filtered_df2

,Scheme,Benefits,SECTOR
0,Small Business Innovation Grant (New),Grant in Aid Up to ₹ 50 Lakhs\n \n \n The Comp...,"[Agriculture, Biotechnology, Pharmaceuticals, ..."
1,Mission AI (Deadline Extended),1. Initial submission and screening (Stage-1):...,"[AI, Healthcare, Agriculture, Education, Envir..."
2,Funding Support (Guj) (New),Grant support: Up to ₹30 lakh\n Assistance to ...,[All Sector]
3,BUILD PROGRAM-2024,· Financial Benefits: Win Grant up to INR 10 L...,"[Technology, Social Impact, Sustainability]"
4,Women Startup Scheme (MH) (Deadline Extended),Grant of up to 25 lakhs,"[Agriculture, Education, Healthcare, Mobility,..."
...,...,...,...
56,Krishi Udaan (New),Pitching for Funds from Angel/VC\n Pitching fo...,"[Agriculture, Food & Beverages, Animal Husbandry]"
57,Agri Commodity Fund,Funding up to ₹ 16 Cr as Loan (RoI and Tenure ...,"[Agriculture, Commodity, Environmental Sustain..."
58,Seed Fund (KL),Seed Fund of Rs.15 Lakhs at a subsidised inter...,[All Sector]
59,Electricity Duty Exemption for Mfg Unit (Guj) ...,Electricity Duty Exemption for the first 5 yea...,[Manufacturing]


In [221]:
filtered_df3 = filtered_df2[filtered_df2["SECTOR"].apply(lambda x: sector in x)][["Scheme" , "Benefits"]]

In [222]:
filtered_df3

,Scheme,Benefits
2,Funding Support (Guj) (New),Grant support: Up to ₹30 lakh\n Assistance to ...
9,Seed Fund (BR),Seed Funding: Up to INR 10 Lakhs per Start-up ...
13,Innovation Grant,1. Idea Grant: Grant of upto INR 3 lakhs.\n 2....
15,Global Impact Grant,Stage 1: Pilot: ₹1.64 Cr.\n Stage 2: Testing ...
17,Excel Your Venture (Top Trending),Funding Opportunities from ₹5L to ₹4Cr (Grant/...
25,Cluster Development Program (CDP),Private Investment Range from ₹ 25 Lakhs to 50...
28,Horizon Fund,"Initial investment of USD 100,000 \n (Up to I..."
39,Hackathon 4.0 (Ever Green Scheme),Grant of up to INR 15 Lakhs
42,SRUJAN - 2024 (Deadline Extended),Grant of 2.5 to 10 Lakhs
43,Universal Grant,Pilot (up to ₹ 1.9 Cr)\n \n Test & Transition ...


In [ ]:

    # filtered_df = stored_df_test_service[(stored_df_test_service["COMPANY TYPE"] == a)][["Scheme" , "Benefits"]] #& (stored_df_test_service["Criteria2"] == b)]
    filtered_df = stored_df_test_service[stored_df_test_service["COMPANY TYPE"].apply(lambda x: selected_value in x)][["Scheme" , "Benefits" , "SECTOR"]]
    filtered_df1 = stored_df_test_service[stored_df_test_service["COMPANY TYPE"].apply(lambda x: selected_value1 in x)][["Scheme" , "Benefits", "SECTOR"]]
    filtered_df2 = pd.concat([filtered_df, filtered_df1]).drop_duplicates().reset_index(drop=True)
    filtered_df3 = filtered_df2[filtered_df2["SECTOR"].apply(lambda x: sector in x)][["Scheme" , "Benefits"]]
    return filtered_df3

In [284]:
stored_df_test_service[stored_df_test_service["Scheme"] == 'CGTMSE (MSME Loan)']

,SR. NO.,Scheme,Eligibility Criteria,COMPANY TYPE,SECTOR,Benefits,Status
75,76,CGTMSE (MSME Loan),Any Company with GST,"[Proprietorship, Partnership, Limited Liabilit...",[All Sectors],Up to 5 CR Loan,Live


C:\Users\Egnio\AppData\Local\Temp\ipykernel_23808\2583179022.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  search_result_df = stored_df_test_service[stored_df_test_service['Scheme'].str.contains(keyword, case=True, na=False)]


In [397]:
filtered_df = stored_df_test_service[["SR. NO.", "Scheme", "Benefits", "SECTOR", "COMPANY TYPE", "Status"]]

In [393]:
filtered_df

,SR. NO.,Scheme,Benefits,SECTOR,COMPANY TYPE,Status
0,1,Hackathon 4.0,Grant of up to INR 15 Lakhs,[All Sector],[ALL],Live
1,2,Small Business Innovation Grant,Grant in Aid Up to ₹ 50 Lakhs\n \n \n The Comp...,"[Agriculture, Biotechnology, Pharmaceuticals, ...","[Private Limited, Limited Liability Partnershi...",Live
3,4,Funding Support (Guj),Grant support: Up to ₹30 lakh\n Assistance to ...,[All Sector],"[Private Limited, Limited Liability Partnershi...",Live
5,6,BUILD PROGRAM-2024,· Financial Benefits: Win Grant up to INR 10 L...,"[Technology, Social Impact, Sustainability]","[Individuals, Proprietorship, Partnership, Lim...",Live
7,8,Sustainable Impact,-Up to INR 10 Lakhs Seed funding in the form o...,"[Renewable Energy, Green Technology, Environme...","[Proprietorship, Limited Liability Partnership...",Live
...,...,...,...,...,...,...
74,75,PMEGP,Service - Up 20 Lakhs & Manufacturing - Up to ...,[All Sectors],"[Individuals, Proprietorship, Partnership, Lim...",Live
75,76,CGTMSE (MSME Loan),Up to 5 CR Loan,[All Sectors],"[Proprietorship, Partnership, Limited Liabilit...",Live
76,77,National Agri Infra Financing Facility,Up to 2 CR Loan (Credit Guarantee Coverage),[Agriculture],"[Individuals, FPO, Proprietorship, Partnership...",Live
77,78,Prime Minister Micro Food Processing Enterpris...,Credit-linked capital subsidy @35% of the elig...,[Food Processing],"[Individuals, Proprietorship, Partnership, Lim...",Live


In [388]:
sector = "All Sector"

In [395]:
filtered_df = filtered_df[(filtered_df["SECTOR"].apply(lambda x: sector in x)) | 
                                  (filtered_df["SECTOR"].apply(lambda x: 'All Sector' in x))]

In [380]:
filtered_df.shape

(65, 6)

In [398]:
selected_value = 'HUF'
filtered_df = filtered_df[(filtered_df["COMPANY TYPE"].apply(lambda x: selected_value in x)) | 
                                  (filtered_df["COMPANY TYPE"].apply(lambda x: 'ALL' in x))]

In [399]:
filtered_df#.shape#["COMPANY TYPE"].value_counts()

,SR. NO.,Scheme,Benefits,SECTOR,COMPANY TYPE,Status
0,1,Hackathon 4.0,Grant of up to INR 15 Lakhs,[All Sector],[ALL],Live
8,9,AI Hackathon,"Cash prizes worth $40,000 (up to ₹33 Lakhs)\n ...","[AI, Manufacturing, Smart City, Education, Hea...",[ALL],Live
14,15,SRUJAN - 2024,Grant of 2.5 to 10 Lakhs,[All Sector],[ALL],Live
21,22,Young Innovators Grant,Up To INR 20 Lakh Grant,[All Sector],[ALL],Live
22,23,Pathfinder Grant,"Grant up to ₹ 12,00,000","[Agriculture, Environment, Education, Social I...",[ALL],Live
55,56,GST Sahay,Invoice based Funding: \n Loan upto 5 Cr\n RoI...,"[Manufacturing, Trade]","[Proprietorship, Partnership, Limited Liabilit...",Live
61,62,Solar Rooftop Scheme,Subsidy: up to 60% of the total cost (up to Rs...,"[Renewable Energy, Residential]","[ALL, Cooperative Societies]",Live
63,64,Capital Subsidy Scheme (TN),"1. Capital Subsidy: 25% , maximum of ₹ 150 Lak...",[Manufacturing],"[Proprietorship, Partnership, Limited Liabilit...",Live
72,73,Startup Veer (MH),Funding up to INR 10 Cr\nBe part of the show\n...,[All Sectors],[ALL],Live


In [522]:
stored_df_test_service[["SR. NO.", "Scheme", "Benefits", "SECTOR", "COMPANY TYPE", "Deadline" , "Days left"]].sort_values(by='Days left', ascending=True)

,SR. NO.,Scheme,Benefits,SECTOR,COMPANY TYPE,Deadline,Days left
46,47,Web3 Cohort,Up to ₹ 1.35 Cr convertible investment\n Up to...,"[Blockchain, Technology, Decentralization]","[Proprietorship, Partnership, Limited Liabilit...",3/10/2024,1
5,6,BUILD PROGRAM-2024,· Financial Benefits: Win Grant up to INR 10 L...,"[Technology, Social Impact, Sustainability]","[Individuals, Proprietorship, Partnership, Lim...",14/10/2024,12
7,8,Sustainable Impact,-Up to INR 10 Lakhs Seed funding in the form o...,"[Renewable Energy, Green Technology, Environme...","[Proprietorship, Limited Liability Partnership...",29/10/2024,27
1,2,Small Business Innovation Grant,Grant in Aid Up to ₹ 50 Lakhs\n \n \n The Comp...,"[Agriculture, Biotechnology, Pharmaceuticals, ...","[Private Limited, Limited Liability Partnershi...",30/10/2024,28
72,73,Startup Veer (MH),Funding up to INR 10 Cr\nBe part of the show\n...,[All Sector],[ALL],30/10/2024,28
69,70,Women-led Venture Program,Funding up to INR 15 Lakhs,[All Sector],"[Individuals, Proprietorship, Partnership, Lim...",30/10/2024,28
36,37,Pre-seed Program,Funding up to $1M (Up to ₹ 8 Cr) in the form o...,"[Education, Fintech, Healthcare, E-commerce, M...","[Limited Liability Partnership (LLP), Private ...",16/11/2024,45
24,25,Pitch-Pilot-Program (3P),"Grant support of up to INR 20 lakh, in three t...","[Water Management, Environmental Sustainability]","[Limited Liability Partnership (LLP), Private ...",20/11/2024,49
41,42,Venture Accelerator,Financial Benefits: Equity-based investment up...,[All Sector],"[Private Limited, Limited Liability Partnershi...",4/12/2024,63
0,1,Hackathon 4.0,Grant of up to INR 15 Lakhs,[All Sector],[ALL],10/10/2024,8


In [527]:
stored_df_test_service[stored_df_test_service["Days left"] == 999]

,SR. NO.,Scheme,Eligibility Criteria,COMPANY TYPE,SECTOR,Benefits,Status,Deadline,Days left,Pamphlet link


In [546]:
import streamlit as st

In [547]:
# Create the Streamlit UI
st.title('Filtered DataFrame')

# Filters
company_type = st.selectbox("Select Company Type", options=['ALL'] + criteria1_options, index=0)
sector = st.selectbox("Select Sector", options=['All Sectors'] + criteria2_options, index=0)

# Apply filters
def filter_dataframe1(selected_value, sector):
    filtered_df = stored_df_test_service.copy()

    # Apply company type filter if not 'ALL'
    if selected_value != 'ALL':
        filtered_df = filtered_df[(filtered_df["COMPANY TYPE"].apply(lambda x: selected_value in x)) | 
                                  (filtered_df["COMPANY TYPE"].apply(lambda x: 'ALL' in x))]

    # Apply sector filter if not 'All Sectors'
    if sector != 'All Sectors':
        filtered_df = filtered_df[(filtered_df["SECTOR"].apply(lambda x: sector in x)) | 
                                  (filtered_df["SECTOR"].apply(lambda x: 'All Sectors' in x))]

    return filtered_df

# Show the filtered DataFrame
filtered_df = filter_dataframe1(company_type, sector)
st.dataframe(filtered_df[["SR. NO.", "Scheme", "Benefits", "SECTOR", "COMPANY TYPE", "Deadline", "Days left"]])

# Search functionality
search_scheme = st.selectbox("Search Scheme", options=criteria3_options)

if search_scheme:
    search_result_df = stored_df_test_service[stored_df_test_service['Scheme'] == search_scheme]
    st.write("Search Results:")
    st.dataframe(search_result_df)

2024-10-02 17:29:33.461 
  command:

    streamlit run C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [548]:
!streamlit run ipykernel_launcher.py

Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Scripts\streamlit-script.py", line 6, in <module>
    from streamlit.cli import main
ModuleNotFoundError: No module named 'streamlit.cli'
